In [1]:
import astropy.io.fits as fits
from astropy.table import Table
import csv
import numpy as np
import sys
import pandas as pd

sys.path.append('../')
from emission_lines import *

In [2]:
# Group emission lines
data = pd.read_csv("../DEM_goodlinelist.csv")
rest_lam_data = pd.DataFrame(data)

# Group emission lines
emission_line_groups = []
grouped_lines = grouping_emission_lines(1160, rest_lam_data)
for ion in grouped_lines:
    for group in grouped_lines[ion]:
        emission_line_groups.append(group)

emission_line_groups

[[1174.935, 1175.265, 1175.592, 1175.713, 1175.713, 1175.989, 1176.372],
 [1206.499],
 [1294.543, 1296.726, 1298.892, 1298.944, 1301.147, 1303.323],
 [1218.39],
 [1371.292],
 [1238.821, 1242.804],
 [1253.811, 1259.519],
 [1260.417, 1264.737, 1265.001],
 [1304.369],
 [1526.706, 1533.43],
 [1323.907, 1323.952],
 [1334.524, 1335.709],
 [1354.08],
 [1393.755, 1402.77],
 [1399.779, 1401.171, 1407.383],
 [1404.818, 1406.016],
 [1486.496],
 [1548.201, 1550.772],
 [1640.42],
 [1666.142]]

## Training Data

In [3]:
flux_array, wavelength_array, emission_line_array, label_array = [], [], [], []

#### GJ-341

In [4]:
spectra = fits.getdata('cnn_spectra/hlsp_muscles_hst_cos_gj_341_g130m_v1_component-spec.fits')
w, f , e = spectra['WAVELENGTH'], spectra['FLUX'], spectra['ERROR']
ecsv_table = Table.read('cnn_spectra_flux/gj-341.ecsv', format='ascii.ecsv', delimiter=' ')

peak_width, peak_width_pixels, flux_range = peak_width_finder('m', w)

noise_indexes = []
for i, error_value in enumerate(ecsv_table['Error']):
    count = i
    # Check if noise
    if error_value == 0:
        label_array = np.append(label_array, 'Noise')
    else:
        label_array = np.append(label_array, 'Not Noise')

    # Find emission line group
    for group in emission_line_groups:
        group_mask = (w > group[0] - peak_width) & (w < group[len(group) - 1] + peak_width) 
        # Check if valid
        if not any(f[group_mask]):
            continue
            
        rest_wavelength = ecsv_table['Rest Wavelength'][i]
        wavelengths = w[group_mask]
        if (rest_wavelength > wavelengths[0]) and (rest_wavelength < wavelengths[len(wavelengths) - 1]):
            flux_array.append(f[group_mask])
            wavelength_array.append(w[group_mask])
            emission_line_array.append(group)
            break